In [57]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import PassiveAggressiveRegressor
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [58]:
import warnings
warnings.filterwarnings('ignore')


In [59]:
path= '../data/bus_stop_times_feature_added_all.csv'
df = pd.read_csv(path)
df = df[df['week_no'] < 25]

def condition(x):
    if x == 0:
        return 0
    else:
        return 1

df = df[df['dwell_time_in_seconds'].apply(condition) == 1]


In [60]:
df = df[['deviceid',
         'bus_stop',
         'week_no',
         'day_of_week',
         'Sunday/holiday',
         'saturday',
         'time_of_day',
         'dt(w-1)',
         'dt(w-2)',
         'dt(w-3)',
         'dt(t-1)',
         'dt(t-2)',
         'dt(n-1)',
         'dt(n-2)',
         'dt(n-3)',
         'temp', 
         'precip',
         'rt(n-1)',
         'dwell_time_in_seconds']]
df

,deviceid,bus_stop,week_no,day_of_week,Sunday/holiday,saturday,time_of_day,dt(w-1),dt(w-2),dt(w-3),dt(t-1),dt(t-2),dt(n-1),dt(n-2),dt(n-3),temp,precip,rt(n-1),dwell_time_in_seconds
0,262,101,1,4,0,0,6.50,92.0,92.0,92.0,92.0,92.0,92.0,92.0,92.0,20.0,0.000,69.0,74.0
2,262,103,1,4,0,0,6.75,28.0,28.0,28.0,28.0,28.0,0.0,74.0,28.0,20.0,0.000,496.0,6.0
4,262,105,1,4,0,0,6.75,230.0,230.0,230.0,230.0,230.0,0.0,6.0,0.0,20.0,0.000,97.0,211.0
5,262,106,1,4,0,0,7.00,24.0,24.0,24.0,24.0,24.0,211.0,0.0,6.0,20.7,0.000,210.0,35.0
6,262,107,1,4,0,0,7.00,15.0,15.0,15.0,15.0,15.0,35.0,211.0,0.0,20.7,0.000,45.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62820,121,110,24,6,1,0,14.25,21.0,21.0,21.0,21.0,21.0,15.0,15.0,15.0,23.7,6.742,190.0,15.0
62821,121,111,24,6,1,0,14.25,10.0,10.0,10.0,10.0,10.0,15.0,15.0,15.0,23.7,6.742,165.0,15.0
62822,121,112,24,6,1,0,14.25,8.0,8.0,8.0,8.0,8.0,15.0,15.0,15.0,23.7,6.742,177.0,6.0
62823,121,113,24,6,1,0,14.25,19.0,19.0,19.0,19.0,15.0,6.0,15.0,15.0,23.7,6.742,168.0,75.0


In [61]:
def predict(model, dt, t):
    preds = model.predict(dt)
    rmse = np.sqrt(mean_squared_error(t['dwell_time_in_seconds'], preds)) 
    mae = mean_absolute_error(t['dwell_time_in_seconds'], preds)
#     # Create an empty DataFrame
#     ddf = pd.DataFrame()

# # Add 'dwell_time_in_seconds' and 'preds' as columns
#     ddf['dwell_time_in_seconds'] = t['dwell_time_in_seconds']
#     ddf['preds'] = preds 
#     ddf.to_csv('new_data.csv', index=False)

    print("MAE (1): %f" % (mae)) 

<hr>

In [62]:
batch = []
dbatch =[]
batch.append(df[df['week_no'] <= 19]) 
for i in range(20,24):
    batch.append(df[df['week_no'] == i])
for i in batch:
    dbatch.append([i.drop(columns=['week_no', 'dwell_time_in_seconds']), i['dwell_time_in_seconds']])

In [63]:
def inc_train(is_initial, data_batch, prev_model):
    if is_initial:
        curr_model = PassiveAggressiveRegressor(C=0.1, max_iter=1000, random_state=42)
        curr_model.fit(data_batch[0], data_batch[1])
    else:
        curr_model = prev_model.partial_fit(data_batch[0], data_batch[1])
    return curr_model

In [64]:
params = {'objective': 'reg:squarederror', 'verbose': False, 'colsample_bytree': 0.7,
    'learning_rate': 0.1,
    'max_depth': 6,
    'alpha': 10,
    'n_estimators': 100}

In [65]:
models = []
for i in range(len(batch)):
    if i!=0:
        curr_model = inc_train(False,  dbatch[i], models[i-1])
    else:
        curr_model = inc_train(True, dbatch[i], None)
    models.append(curr_model)
    for j in range(len(batch)):
        print(f"Results of data batch {j+1}  with model {i+1}")
        predict(curr_model, dbatch[j][0], batch[j])

Results of data batch 1  with model 1
MAE (1): 102.797744
Results of data batch 2  with model 1
MAE (1): 111.080709
Results of data batch 3  with model 1
MAE (1): 104.963425
Results of data batch 4  with model 1
MAE (1): 115.615476
Results of data batch 5  with model 1
MAE (1): 100.129813
Results of data batch 1  with model 2
MAE (1): 108.674341
Results of data batch 2  with model 2
MAE (1): 117.533557
Results of data batch 3  with model 2
MAE (1): 110.972933
Results of data batch 4  with model 2
MAE (1): 122.777671
Results of data batch 5  with model 2
MAE (1): 108.116786
Results of data batch 1  with model 3
MAE (1): 47.608586
Results of data batch 2  with model 3
MAE (1): 47.596548
Results of data batch 3  with model 3
MAE (1): 48.610167
Results of data batch 4  with model 3
MAE (1): 47.271312
Results of data batch 5  with model 3
MAE (1): 45.221544
Results of data batch 1  with model 4
MAE (1): 30.879762
Results of data batch 2  with model 4
MAE (1): 30.589909
Results of data batch